<a href="https://colab.research.google.com/github/willwang171/march_madness_sentiment/blob/main/final_model_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pypdf
!pip install langchain
!pip install chromadb
!pip install pytest

In [ ]:
!pip install langchain-chroma

In [ ]:
!pip install langchain-text-splitters tiktoken


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_path="/content/drive/MyDrive/bert_finetuned_model"

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# Assuming the RecursiveCharacterTextSplitter is correctly set up for tiktoken
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-3.5-turbo",  # Assuming using GPT-4
    chunk_size=100,  # Number of tokens per chunk
    chunk_overlap=10  # Number of tokens to overlap; adjust as needed
)

# Function to apply text splitting to each comment
def split_comments(comments):
    split_texts = []
    for comment in comments:
        chunks = text_splitter.split_text(comment)
        split_texts.append(chunks)
    return split_texts

# Splitting the DataFrame's comments
second_dv['split_comments'] = split_comments(second_dv['Comment'])

SyntaxError: f-string: unmatched '[' (<ipython-input-101-51e827b453c2>, line 1)

In [ ]:
second_dv=pd.read_csv("/content/game_thread_13_louisiana_4_tennessee_0940_pm_et.csv")


In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from langchain_chroma import Chroma

In [ ]:


class CustomBERTEmbeddings:
    def __init__(self, model_path):
        self.tokenizer = BertTokenizer.from_pretrained(model_path)
        self.model = BertModel.from_pretrained(model_path)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def __call__(self, text: str) -> list:
        return self.embed_documents([text])

    def embed_documents(self, texts):
        embeddings = []
        for text in texts:
            encoded_input = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
            encoded_input = {k: v.to(self.device) for k, v in encoded_input.items()}
            with torch.no_grad():
                output = self.model(**encoded_input)
                embeddings = output.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
        return embeddings.tolist()  # Ensure output is a list

    def embed_text(self, text: str) -> np.ndarray:
        encoded_input = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        encoded_input = {k: v.to(self.device) for k, v in encoded_input.items()}
        with torch.no_grad():
            output = self.model(**encoded_input)
            embeddings = output.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
        return embeddings.tolist()

    def embed_query(self, query: str) -> np.ndarray:
        # This function is specifically for embedding queries if needed by the database
        return self.embed_text(query)




In [ ]:
embedding_function = CustomBERTEmbeddings(model_path)


In [ ]:
df=pd.read_csv("/content/game_thread_14_grand_canyon_3_gonzaga_1935_pm_et.csv.csv")

In [ ]:
second_dv=pd.read_csv("hello.csv")

In [ ]:

# Initialize Chroma database
chroma_db = Chroma('chroma_collection', embedding_function=embedding_function)

all_embeddings = {}
i=0
def upload_to_chroma(df, chroma_db):
    for index, row in df.iterrows():
        for comment in row['split_comments']:
            embedding = [embedding_function(comment)]
            if isinstance(embedding, np.ndarray):
                embedding = [embedding.tolist()]
            embedding_id = str(index) + '_' + str(comment)  # Example ID generation
            all_embeddings[embedding_id] = embedding
            metadata = {'embedding_id': embedding_id}
            print("Uploading text with ID:", embedding_id)
            print("Embedding type:", type(embedding))
            print("Embedding length:", len(embedding))
            chroma_db.add_texts([comment], metadata=[metadata])

# Assuming df is already loaded and contains the 'split_comments' as processed text
upload_to_chroma(second_dv.sample(500), chroma_db)


Uploading text with ID: 591_I’m not opposed to this
Embedding type: <class 'list'>
Embedding length: 1
Uploading text with ID: 745_This is a good night!

Edit: jk but yes it is.
Embedding type: <class 'list'>
Embedding length: 1
Uploading text with ID: 552_This is so over
Embedding type: <class 'list'>
Embedding length: 1
Uploading text with ID: 796_They’re gonna win bro
Embedding type: <class 'list'>
Embedding length: 1
Uploading text with ID: 822_They have had to completely change the offense approach up since losing Ziegler. It’s a gigantic loss for them. They’re scraping together an offense but without him it’s pretty mediocre.
Embedding type: <class 'list'>
Embedding length: 1
Uploading text with ID: 475_You have got to be kidding me. All Tennessee does is bitch and moan to the refs. I would be embarrassed as a fan.
Embedding type: <class 'list'>
Embedding length: 1
Uploading text with ID: 291_Refs bailing out the Vols hardcore now
Embedding type: <class 'list'>
Embedding length: 

In [ ]:
import os
from getpass import getpass

# Ensure the Gradient access token and workspace ID are set
if not os.environ.get("GRADIENT_ACCESS_TOKEN", None):
    os.environ["GRADIENT_ACCESS_TOKEN"] = getpass("Enter your Gradient access token:")
if not os.environ.get("GRADIENT_WORKSPACE_ID", None):
    os.environ["GRADIENT_WORKSPACE_ID"] = getpass("Enter your Gradient workspace ID:")


In [ ]:
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)

In [ ]:
!pip install langchain_experimental

In [ ]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

In [ ]:
python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)



'2\n'

In [ ]:
df.dropna(inplace=True)

In [ ]:
# Function to query Chroma and then use Gradient model to generate an answer
def query_and_generate_answer(question, chroma_db, llm_chain):
    # Query Chroma database
    results = chroma_db.similarity_search_with_score(question, k=30)
    sentiments=[]

    game="[Game: grand canyon vs gonzaga]"
    for com in results:
        try:
          row = df[df['Comment'] == com[0].page_content]
          sentimentscore=row['sentiment_score'].values[0]
          sentiments.append(sentimentscore)
        except:
          pass
    full_str=''
    # [Sentiment = snescore] comment
    summer=sum(sentiments)
    for i in range(len(sentiments)):
      con =f"[Game: grand canyon vs gonzaga] [Sentiment: {float(sentiments[i])}] {results[i][0].page_content}"
      full_str=full_str+con
    full_str=full_str+f"QUESTION:  what are the chances of gonzaga winning?  ANSWER:"
    print(full_str)
    response = llm_chain.run(question=full_str)
    return response
# Set up LLM Chain
from langchain.chains import LLMChain
from langchain.llms import GradientLLM
from langchain.prompts import PromptTemplate



# Setup LLM Chain
prompt_template = """
Question: {question} Answer:
"""
prompt = PromptTemplate.from_template(prompt_template)

llm = GradientLLM(model="88238525-5c2e-4bf7-bed7-1b15b14b7f4b_model_adapter", model_kwargs={"max_generated_token_count": 128})

# Setup LLM Chainm
#QUESTION: What's the chance northwestern beats boise state based on fan sentiment comments? in percent?

question = "What are the chances of gonzaga winning vs grand canyon?"


llm_chain = LLMChain(prompt=prompt, llm=llm)

answer = query_and_generate_answer(question,  chroma_db, llm_chain)
print(answer)
#91% gonzaga vs 9%

In [ ]:
# Function to query Chroma and then use Gradient model to generate an answer
def query_and_generate_answer(question, chroma_db, llm_chain):
    # Query Chroma database
    results = chroma_db.similarity_search_with_score(question, k=30)
    sentiments=[]

    game="[Game: louisiana vs tennessee]"

    for com in results:
        try:
          row = df[df['Comment'] == com[0].page_content]
          sentimentscore=row['sentiment_score'].values[0]
          sentiments.append(sentimentscore)
        except:
          pass
    full_str=''
    # [Sentiment = snescore] comment
    summer=sum(sentiments)
    for i in range(len(sentiments)):
      con =f"{game} [Sentiment: {float(sentiments[i])}] {results[i][0].page_content}"
      full_str=full_str+con
    full_str=full_str+f"QUESTION:  what are the chances of louisiana winning?  ANSWER:"
    print(full_str)
    response = llm_chain.run(question=full_str)
    return response
# Set up LLM Chain
from langchain.chains import LLMChain
from langchain.llms import GradientLLM
from langchain.prompts import PromptTemplate



# Setup LLM Chain
prompt_template = """
Question: {question} Answer:
"""
prompt = PromptTemplate.from_template(prompt_template)

llm = GradientLLM(model="88238525-5c2e-4bf7-bed7-1b15b14b7f4b_model_adapter", model_kwargs={"max_generated_token_count": 128})

# Setup LLM Chainm
#QUESTION: What's the chance northwestern beats boise state based on fan sentiment comments? in percent?

question = "What are the chances of louisianna winning vs tennessee?"


llm_chain = LLMChain(prompt=prompt, llm=llm)

answer = query_and_generate_answer(question,  chroma_db, llm_chain)
print(answer)


In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = "Tell me a {adjective} joke"
prompt = PromptTemplate(
    input_variables=["adjective"], template=prompt_template
)
llm_chain = LLMChain(prompt=prompt, llm=llm)

chain = prompt | llm

chain.invoke("your adjective here")

SyntaxError: invalid syntax (<ipython-input-83-786f6167d7a2>, line 1)

In [ ]:
for r in results:
  print(r[0].page_content)

How do they keep missing those lmao
Joining late, wildcat bros, what's going down?
being named northwestern means we’re directionally challenged so we assumed sacramento was in the pacific northwest
it’s so weird the northwestern defense has been good right until the end and I worry that may fade in crunch time
Is the US military down bad on submarines or something?
